# Task 4.10

## Table of Contents

1. Import Libraries + Data
2. Address PII Data
3. Segment Data Regionally
    - is there a difference between type of spender and region?
4. Customer Activity Exclusion Flag

# 01. Import Libraries + Data

In [1]:
# Libraries
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

In [2]:
# define path
path = r'/Users/alliecarlisle/Desktop/CareerFoundry_Data_Analytics/Instacart Basket Analysis'

In [3]:
# final dataframe
df_final = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'final_merge.pkl'))

In [4]:
df_final.shape

(30328763, 30)

In [5]:
df_final.head(5)

,user_id,first_name,last_name,state,age,n_dependants,family_status,income,order_id,order_number,...,price_range_loc,busiest_days,busiest_time_of_day,max_order,loyalty_flag,mean_price,spending_flag,median_days,frequency_flag,_merge
0,26711,Deborah,Esquivel,Missouri,48,3,married,165665,2543867,5,...,Mid-range product,Busiest days,busiest time of day,8,New customer,8.205882,Low spender,19.0,Regular customer,both
1,26711,Deborah,Esquivel,Missouri,48,3,married,165665,1285508,7,...,Mid-range product,Regular days,busiest time of day,8,New customer,8.205882,Low spender,19.0,Regular customer,both
2,26711,Deborah,Esquivel,Missouri,48,3,married,165665,2578584,8,...,Mid-range product,Busiest days,busiest time of day,8,New customer,8.205882,Low spender,19.0,Regular customer,both
3,26711,Deborah,Esquivel,Missouri,48,3,married,165665,423547,2,...,Mid-range product,Regular days,busiest time of day,8,New customer,8.205882,Low spender,19.0,Regular customer,both
4,26711,Deborah,Esquivel,Missouri,48,3,married,165665,2524893,3,...,Mid-range product,Slowest days,busiest time of day,8,New customer,8.205882,Low spender,19.0,Regular customer,both


In [6]:
# display all columns
pd.options.display.max_columns = None

In [7]:
df_final.columns

Index(['user_id', 'first_name', 'last_name', 'state', 'age', 'n_dependants',
       'family_status', 'income', 'order_id', 'order_number',
       'orders_day_of_the_week', 'order_hour_of_day', 'time_between_orders',
       'product_id', 'add_to_cart_order', 'reordered', 'product_name',
       'aisle_id', 'department_id', 'prices', 'price_range_loc',
       'busiest_days', 'busiest_time_of_day', 'max_order', 'loyalty_flag',
       'mean_price', 'spending_flag', 'median_days', 'frequency_flag',
       '_merge'],
      dtype='object')

In [8]:
df_final.head(3)

,user_id,first_name,last_name,state,age,n_dependants,family_status,income,order_id,order_number,orders_day_of_the_week,order_hour_of_day,time_between_orders,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,price_range_loc,busiest_days,busiest_time_of_day,max_order,loyalty_flag,mean_price,spending_flag,median_days,frequency_flag,_merge
0,26711,Deborah,Esquivel,Missouri,48,3,married,165665,2543867,5,1,9,30.0,196,2,0,Soda,77,7,9.0,Mid-range product,Busiest days,busiest time of day,8,New customer,8.205882,Low spender,19.0,Regular customer,both
1,26711,Deborah,Esquivel,Missouri,48,3,married,165665,1285508,7,5,15,11.0,196,1,1,Soda,77,7,9.0,Mid-range product,Regular days,busiest time of day,8,New customer,8.205882,Low spender,19.0,Regular customer,both
2,26711,Deborah,Esquivel,Missouri,48,3,married,165665,2578584,8,1,15,10.0,196,2,1,Soda,77,7,9.0,Mid-range product,Busiest days,busiest time of day,8,New customer,8.205882,Low spender,19.0,Regular customer,both


# 02. Address PII Data

In [9]:
# sensitive data in first_name and last_name columns
df_final = df_final.drop(columns=['first_name', 'last_name'])

In [10]:
# check
df_final.head(3)

,user_id,state,age,n_dependants,family_status,income,order_id,order_number,orders_day_of_the_week,order_hour_of_day,time_between_orders,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,price_range_loc,busiest_days,busiest_time_of_day,max_order,loyalty_flag,mean_price,spending_flag,median_days,frequency_flag,_merge
0,26711,Missouri,48,3,married,165665,2543867,5,1,9,30.0,196,2,0,Soda,77,7,9.0,Mid-range product,Busiest days,busiest time of day,8,New customer,8.205882,Low spender,19.0,Regular customer,both
1,26711,Missouri,48,3,married,165665,1285508,7,5,15,11.0,196,1,1,Soda,77,7,9.0,Mid-range product,Regular days,busiest time of day,8,New customer,8.205882,Low spender,19.0,Regular customer,both
2,26711,Missouri,48,3,married,165665,2578584,8,1,15,10.0,196,2,1,Soda,77,7,9.0,Mid-range product,Busiest days,busiest time of day,8,New customer,8.205882,Low spender,19.0,Regular customer,both


# 03. Segment the Data Regionally

In [11]:
# derive a new column for region from "state"
result = []

for value in df_final['state']:
    if value in ['Maine', 'New Hampshire', 'Vermont', 'Massachusetts', 'Rhode Island', 'Connecticut', 'New York', 
                'Pennsylvania', 'New Jersey']:
        result.append('Northeast')
    elif value in ['Wisconsin', 'Michigan', 'Illinois', 'Indiana', 'Ohio', 'North Dakota', 'South Dakota', 
                  'Nebraska', 'Kansas', 'Minnesota', 'Iowa', 'Missouri']:
        result.append('Midwest')
    elif value in ['Delaware', 'Maryland', 'District of Columbia', 'Virginia', 'West Virginia', 'North Carolina', 
                  'South Carolina', 'Georgia', 'Florida', 'Kentucky', 'Tennessee', 'Mississippi', 'Alabama', 
                  'Oklahoma', 'Texas', 'Arkansas', 'Louisiana']:
        result.append('South')
    else:
        result.append('West')

In [12]:
df_final['region'] = result

In [13]:
df_final['region'].value_counts(dropna = False)

South        10099817
West          7765088
Midwest       7107586
Northeast     5356272
Name: region, dtype: int64

### I added the region values from the value count and compared it to the rows in df_final shape and got 30,328,763 for both.

# 03. Is there a difference in spending habits b/w U.S. regions?

In [22]:
# check head
df_final.head(4)

,user_id,state,age,n_dependants,family_status,income,order_id,order_number,orders_day_of_the_week,order_hour_of_day,time_between_orders,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,price_range_loc,busiest_days,busiest_time_of_day,max_order,loyalty_flag,mean_price,spending_flag,median_days,frequency_flag,_merge,region,exclusion_flag
0,26711,Missouri,48,3,married,165665,2543867,5,1,9,30.0,196,2,0,Soda,77,7,9.0,Mid-range product,Busiest days,busiest time of day,8,New customer,8.205882,Low spender,19.0,Regular customer,both,Midwest,No
1,26711,Missouri,48,3,married,165665,1285508,7,5,15,11.0,196,1,1,Soda,77,7,9.0,Mid-range product,Regular days,busiest time of day,8,New customer,8.205882,Low spender,19.0,Regular customer,both,Midwest,No
2,26711,Missouri,48,3,married,165665,2578584,8,1,15,10.0,196,2,1,Soda,77,7,9.0,Mid-range product,Busiest days,busiest time of day,8,New customer,8.205882,Low spender,19.0,Regular customer,both,Midwest,No
3,26711,Missouri,48,3,married,165665,423547,2,2,9,14.0,38928,1,0,0% Greek Strained Yogurt,120,16,12.6,Mid-range product,Regular days,busiest time of day,8,New customer,8.205882,Low spender,19.0,Regular customer,both,Midwest,No


In [26]:
# compare spending flag and region column
crosstab_2 = pd.crosstab(df_final['region'], df_final['spending_flag'], dropna = False)

In [28]:
# sending this comparison to excel to create a visual there
crosstab_2.to_clipboard()

### Observations - yes, there is a difference regionally, but only within the low spender type. High spenders hover around the same number between regions. Low spenders are the highest in the South and lowest in the Northeast.

# 04. Customer Activity Exclusion Flag

In [14]:
# create new column "exclusion_flag" based on max_order
df_final.loc[df_final['max_order']>=5, 'exclusion_flag'] = 'No'
df_final.loc[df_final['max_order']<5, 'exclusion_flag'] = 'Yes'

In [15]:
# check
df_final['exclusion_flag'].value_counts(dropna = False)

No     29303281
Yes     1025482
Name: exclusion_flag, dtype: int64

In [16]:
# create subset excluding low max_order customers
df_final_high = df_final[df_final['exclusion_flag'] == 'No']

In [17]:
# check
df_final_high.head(5)

,user_id,state,age,n_dependants,family_status,income,order_id,order_number,orders_day_of_the_week,order_hour_of_day,time_between_orders,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,price_range_loc,busiest_days,busiest_time_of_day,max_order,loyalty_flag,mean_price,spending_flag,median_days,frequency_flag,_merge,region,exclusion_flag
0,26711,Missouri,48,3,married,165665,2543867,5,1,9,30.0,196,2,0,Soda,77,7,9.0,Mid-range product,Busiest days,busiest time of day,8,New customer,8.205882,Low spender,19.0,Regular customer,both,Midwest,No
1,26711,Missouri,48,3,married,165665,1285508,7,5,15,11.0,196,1,1,Soda,77,7,9.0,Mid-range product,Regular days,busiest time of day,8,New customer,8.205882,Low spender,19.0,Regular customer,both,Midwest,No
2,26711,Missouri,48,3,married,165665,2578584,8,1,15,10.0,196,2,1,Soda,77,7,9.0,Mid-range product,Busiest days,busiest time of day,8,New customer,8.205882,Low spender,19.0,Regular customer,both,Midwest,No
3,26711,Missouri,48,3,married,165665,423547,2,2,9,14.0,38928,1,0,0% Greek Strained Yogurt,120,16,12.6,Mid-range product,Regular days,busiest time of day,8,New customer,8.205882,Low spender,19.0,Regular customer,both,Midwest,No
4,26711,Missouri,48,3,married,165665,2524893,3,3,11,30.0,38928,1,1,0% Greek Strained Yogurt,120,16,12.6,Mid-range product,Slowest days,busiest time of day,8,New customer,8.205882,Low spender,19.0,Regular customer,both,Midwest,No


In [18]:
# export to pickle
df_final_high.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'df_final_high_orders.pkl'))